# Kernel Methods Tutorial
## CMSC 173 - Machine Learning

This notebook accompanies the kernel methods slides and provides hands-on implementation and visualization of key concepts.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC, SVR
from sklearn.datasets import make_classification, make_regression, make_circles, make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Introduction & Motivation

Kernel methods solve the fundamental problem of learning non-linear patterns in data by implicitly mapping features to higher-dimensional spaces. This section demonstrates why we need kernel methods through visualization of linear vs non-linear separability.

In [ ]:
# Create linearly separable data
X_linear, y_linear = make_classification(n_samples=200, n_features=2, n_redundant=0, 
                                        n_informative=2, n_clusters_per_class=1, 
                                        class_sep=2, random_state=42)

# Create non-linearly separable data
X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.3, random_state=42)
X_moons, y_moons = make_moons(n_samples=200, noise=0.1, random_state=42)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot linearly separable data
axes[0].scatter(X_linear[y_linear==0, 0], X_linear[y_linear==0, 1], c='red', alpha=0.7, label='Class 0')
axes[0].scatter(X_linear[y_linear==1, 0], X_linear[y_linear==1, 1], c='blue', alpha=0.7, label='Class 1')
axes[0].set_title('Linearly Separable')
axes[0].legend()

# Plot circles data
axes[1].scatter(X_circles[y_circles==0, 0], X_circles[y_circles==0, 1], c='red', alpha=0.7, label='Class 0')
axes[1].scatter(X_circles[y_circles==1, 0], X_circles[y_circles==1, 1], c='blue', alpha=0.7, label='Class 1')
axes[1].set_title('Circles (Non-linear)')
axes[1].legend()

# Plot moons data
axes[2].scatter(X_moons[y_moons==0, 0], X_moons[y_moons==0, 1], c='red', alpha=0.7, label='Class 0')
axes[2].scatter(X_moons[y_moons==1, 0], X_moons[y_moons==1, 1], c='blue', alpha=0.7, label='Class 1')
axes[2].set_title('Moons (Non-linear)')
axes[2].legend()

plt.tight_layout()
plt.show()

## 2. Support Vector Machines - Linear Case

Support Vector Machines find the optimal decision boundary by maximizing the margin between classes. This section demonstrates the fundamental SVM concepts using linearly separable data.

In [ ]:
def plot_svm_decision_boundary(X, y, model, title):
    """Plot SVM decision boundary with support vectors highlighted."""
    plt.figure(figsize=(10, 6))
    
    # Create mesh for decision boundary
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Predict on mesh
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot decision boundary
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
    
    # Plot data points
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    
    # Highlight support vectors
    plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1],
                s=100, facecolors='none', edgecolors='green', linewidths=2,
                label=f'Support Vectors ({len(model.support_vectors_)})')
    
    plt.title(f'{title} - Margin: {2/np.linalg.norm(model.coef_):.3f}')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.show()

# Train linear SVM on linearly separable data
svm_linear = SVC(kernel='linear', C=1.0)
svm_linear.fit(X_linear, y_linear)

plot_svm_decision_boundary(X_linear, y_linear, svm_linear, 'Linear SVM')

## 3. Hard vs Soft Margin

Real-world data is rarely perfectly separable. Soft margin SVM allows some misclassifications by introducing slack variables, controlled by the regularization parameter C.

In [ ]:
# Add noise to make data not perfectly separable
X_noisy = X_linear + np.random.normal(0, 0.3, X_linear.shape)

# Compare different C values
C_values = [0.1, 1.0, 10.0]
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, C in enumerate(C_values):
    svm = SVC(kernel='linear', C=C)
    svm.fit(X_noisy, y_linear)
    
    # Create mesh
    h = 0.02
    x_min, x_max = X_noisy[:, 0].min() - 1, X_noisy[:, 0].max() + 1
    y_min, y_max = X_noisy[:, 1].min() - 1, X_noisy[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[i].contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
    axes[i].scatter(X_noisy[:, 0], X_noisy[:, 1], c=y_linear, cmap=plt.cm.RdYlBu, edgecolors='black')
    axes[i].scatter(svm.support_vectors_[:, 0], svm.support_vectors_[:, 1],
                   s=100, facecolors='none', edgecolors='green', linewidths=2)
    
    accuracy = svm.score(X_noisy, y_linear)
    axes[i].set_title(f'C = {C}, Accuracy: {accuracy:.3f}\nSupport Vectors: {len(svm.support_vectors_)}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 4. The Kernel Trick

The kernel trick allows SVMs to find non-linear decision boundaries by implicitly mapping data to higher-dimensional spaces. This section demonstrates how different kernels handle non-linear data.

In [ ]:
def compare_kernels(X, y, title_prefix):
    """Compare different kernel functions on the same dataset."""
    kernels = ['linear', 'poly', 'rbf', 'sigmoid']
    kernel_names = ['Linear', 'Polynomial (degree=3)', 'RBF (γ=auto)', 'Sigmoid']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    for i, (kernel, name) in enumerate(zip(kernels, kernel_names)):
        if kernel == 'poly':
            svm = SVC(kernel=kernel, degree=3, C=1.0)
        else:
            svm = SVC(kernel=kernel, C=1.0)
        
        svm.fit(X, y)
        
        # Create mesh
        h = 0.02
        x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
        y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        
        Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        axes[i].contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
        axes[i].scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
        axes[i].scatter(svm.support_vectors_[:, 0], svm.support_vectors_[:, 1],
                       s=100, facecolors='none', edgecolors='green', linewidths=2)
        
        accuracy = svm.score(X, y)
        axes[i].set_title(f'{name}\nAccuracy: {accuracy:.3f}, SVs: {len(svm.support_vectors_)}')
        axes[i].set_xlabel('Feature 1')
        axes[i].set_ylabel('Feature 2')
    
    plt.suptitle(f'{title_prefix} - Kernel Comparison', fontsize=16)
    plt.tight_layout()
    plt.show()

# Compare kernels on circles dataset
compare_kernels(X_circles, y_circles, 'Circles Dataset')

In [ ]:
# Compare kernels on moons dataset
compare_kernels(X_moons, y_moons, 'Moons Dataset')

## 5. RBF Kernel Parameter Tuning

The RBF (Radial Basis Function) kernel is the most popular kernel for SVMs. Its performance depends heavily on two parameters: C (regularization) and γ (kernel coefficient). This section explores their effects.

In [ ]:
# Parameter grid for RBF kernel
C_range = [0.1, 1, 10, 100]
gamma_range = [0.001, 0.01, 0.1, 1]

fig, axes = plt.subplots(len(gamma_range), len(C_range), figsize=(20, 16))

for i, gamma in enumerate(gamma_range):
    for j, C in enumerate(C_range):
        svm = SVC(kernel='rbf', C=C, gamma=gamma)
        svm.fit(X_circles, y_circles)
        
        # Create mesh
        h = 0.02
        x_min, x_max = X_circles[:, 0].min() - 0.5, X_circles[:, 0].max() + 0.5
        y_min, y_max = X_circles[:, 1].min() - 0.5, X_circles[:, 1].max() + 0.5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        
        Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        axes[i, j].contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
        axes[i, j].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, 
                          cmap=plt.cm.RdYlBu, edgecolors='black', s=30)
        
        accuracy = svm.score(X_circles, y_circles)
        axes[i, j].set_title(f'C={C}, γ={gamma}\nAcc: {accuracy:.3f}')
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])

plt.suptitle('RBF Kernel: Effect of C and γ Parameters', fontsize=16)
plt.tight_layout()
plt.show()

## 6. Multi-class Classification

SVMs are inherently binary classifiers. For multi-class problems, we use strategies like One-vs-Rest (OvR) and One-vs-One (OvO). This section compares these approaches.

In [ ]:
# Create multi-class dataset
X_multi, y_multi = make_classification(n_samples=300, n_features=2, n_classes=4, 
                                      n_redundant=0, n_informative=2, 
                                      n_clusters_per_class=1, class_sep=1.5, 
                                      random_state=42)

# Compare multi-class strategies
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Native multi-class SVM
svm_native = SVC(kernel='rbf', C=1, gamma='scale', decision_function_shape='ovr')
svm_native.fit(X_multi, y_multi)

# One-vs-Rest
svm_ovr = OneVsRestClassifier(SVC(kernel='rbf', C=1, gamma='scale'))
svm_ovr.fit(X_multi, y_multi)

# One-vs-One
svm_ovo = OneVsOneClassifier(SVC(kernel='rbf', C=1, gamma='scale'))
svm_ovo.fit(X_multi, y_multi)

models = [svm_native, svm_ovr, svm_ovo]
titles = ['Native Multi-class', 'One-vs-Rest', 'One-vs-One']

for i, (model, title) in enumerate(zip(models, titles)):
    # Create mesh
    h = 0.02
    x_min, x_max = X_multi[:, 0].min() - 1, X_multi[:, 0].max() + 1
    y_min, y_max = X_multi[:, 1].min() - 1, X_multi[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    axes[i].contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Set3)
    scatter = axes[i].scatter(X_multi[:, 0], X_multi[:, 1], c=y_multi, 
                             cmap=plt.cm.Set3, edgecolors='black')
    
    accuracy = model.score(X_multi, y_multi)
    axes[i].set_title(f'{title}\nAccuracy: {accuracy:.3f}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 7. Support Vector Regression (SVR)

Support Vector Regression extends SVM concepts to regression problems. It uses ε-insensitive loss function and can handle non-linear relationships using kernels.

In [ ]:
# Generate regression data with noise
def generate_regression_data(n_samples=100, noise=0.3):
    X = np.linspace(0, 4, n_samples).reshape(-1, 1)
    y = np.sin(2 * X).ravel() + np.random.normal(0, noise, X.shape[0])
    return X, y

X_reg, y_reg = generate_regression_data()

# Compare different SVR kernels
kernels = ['linear', 'poly', 'rbf']
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Create fine grid for smooth curves
X_plot = np.linspace(0, 4, 300).reshape(-1, 1)

for i, kernel in enumerate(kernels):
    if kernel == 'poly':
        svr = SVR(kernel=kernel, degree=3, C=100, epsilon=0.1)
    else:
        svr = SVR(kernel=kernel, C=100, epsilon=0.1)
    
    svr.fit(X_reg, y_reg)
    y_pred = svr.predict(X_plot)
    
    axes[i].scatter(X_reg, y_reg, alpha=0.6, label='Data')
    axes[i].plot(X_plot, y_pred, color='red', linewidth=2, label='SVR prediction')
    axes[i].plot(X_plot, np.sin(2 * X_plot).ravel(), color='green', 
                linewidth=1, linestyle='--', label='True function')
    
    # Highlight support vectors
    axes[i].scatter(X_reg[svr.support_], y_reg[svr.support_], 
                   s=100, facecolors='none', edgecolors='orange', 
                   linewidths=2, label=f'Support Vectors ({len(svr.support_)})')
    
    score = svr.score(X_reg, y_reg)
    axes[i].set_title(f'{kernel.upper()} SVR\nR² Score: {score:.3f}')
    axes[i].set_xlabel('X')
    axes[i].set_ylabel('y')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Epsilon Parameter in SVR

The ε parameter in SVR defines the margin of tolerance where no penalty is given to errors. Points within this margin are not considered support vectors.

In [ ]:
# Effect of epsilon parameter
epsilons = [0.01, 0.1, 0.5, 1.0]
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, eps in enumerate(epsilons):
    svr = SVR(kernel='rbf', C=100, epsilon=eps, gamma='scale')
    svr.fit(X_reg, y_reg)
    y_pred = svr.predict(X_plot)
    
    axes[i].scatter(X_reg, y_reg, alpha=0.6, label='Data')
    axes[i].plot(X_plot, y_pred, color='red', linewidth=2, label='SVR prediction')
    
    # Show epsilon tube
    axes[i].fill_between(X_plot.ravel(), y_pred - eps, y_pred + eps, 
                        alpha=0.2, color='red', label=f'ε-tube (ε={eps})')
    
    # Highlight support vectors
    axes[i].scatter(X_reg[svr.support_], y_reg[svr.support_], 
                   s=100, facecolors='none', edgecolors='orange', linewidths=2)
    
    score = svr.score(X_reg, y_reg)
    axes[i].set_title(f'ε = {eps}\nSupport Vectors: {len(svr.support_)}\nR² Score: {score:.3f}')
    axes[i].set_xlabel('X')
    axes[i].set_ylabel('y')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Kernel Ridge Regression Comparison

Kernel Ridge Regression is another kernel method for regression that uses L2 regularization instead of the ε-insensitive loss. This section compares it with SVR.

In [ ]:
from sklearn.kernel_ridge import KernelRidge

# Compare SVR vs Kernel Ridge Regression
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# SVR
svr = SVR(kernel='rbf', C=100, epsilon=0.1, gamma='scale')
svr.fit(X_reg, y_reg)
y_svr = svr.predict(X_plot)

# Kernel Ridge Regression
krr = KernelRidge(kernel='rbf', alpha=0.01, gamma=None)
krr.fit(X_reg, y_reg)
y_krr = krr.predict(X_plot)

# Plot SVR
axes[0].scatter(X_reg, y_reg, alpha=0.6, label='Data')
axes[0].plot(X_plot, y_svr, color='red', linewidth=2, label='SVR')
axes[0].plot(X_plot, np.sin(2 * X_plot).ravel(), color='green', 
            linewidth=1, linestyle='--', label='True function')
axes[0].scatter(X_reg[svr.support_], y_reg[svr.support_], 
               s=100, facecolors='none', edgecolors='orange', linewidths=2,
               label=f'Support Vectors ({len(svr.support_)})')
axes[0].set_title(f'Support Vector Regression\nR² Score: {svr.score(X_reg, y_reg):.3f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Kernel Ridge
axes[1].scatter(X_reg, y_reg, alpha=0.6, label='Data')
axes[1].plot(X_plot, y_krr, color='blue', linewidth=2, label='Kernel Ridge')
axes[1].plot(X_plot, np.sin(2 * X_plot).ravel(), color='green', 
            linewidth=1, linestyle='--', label='True function')
axes[1].set_title(f'Kernel Ridge Regression\nR² Score: {krr.score(X_reg, y_reg):.3f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

for ax in axes:
    ax.set_xlabel('X')
    ax.set_ylabel('y')

plt.tight_layout()
plt.show()

## 10. Hyperparameter Optimization

Finding optimal hyperparameters is crucial for SVM performance. This section demonstrates systematic parameter tuning using grid search.

In [ ]:
# Grid search for optimal parameters
from sklearn.model_selection import validation_curve

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_circles, y_circles, 
                                                    test_size=0.3, random_state=42)

# Validation curve for C parameter
C_range = np.logspace(-2, 3, 10)
train_scores_C, val_scores_C = validation_curve(
    SVC(kernel='rbf', gamma='scale'), X_train, y_train, 
    param_name='C', param_range=C_range, cv=5, scoring='accuracy'
)

# Validation curve for gamma parameter
gamma_range = np.logspace(-4, 1, 10)
train_scores_gamma, val_scores_gamma = validation_curve(
    SVC(kernel='rbf', C=1.0), X_train, y_train, 
    param_name='gamma', param_range=gamma_range, cv=5, scoring='accuracy'
)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot C parameter validation curve
train_mean_C = np.mean(train_scores_C, axis=1)
train_std_C = np.std(train_scores_C, axis=1)
val_mean_C = np.mean(val_scores_C, axis=1)
val_std_C = np.std(val_scores_C, axis=1)

axes[0].semilogx(C_range, train_mean_C, 'o-', color='blue', label='Training accuracy')
axes[0].fill_between(C_range, train_mean_C - train_std_C, train_mean_C + train_std_C, 
                    alpha=0.2, color='blue')
axes[0].semilogx(C_range, val_mean_C, 'o-', color='red', label='Validation accuracy')
axes[0].fill_between(C_range, val_mean_C - val_std_C, val_mean_C + val_std_C, 
                    alpha=0.2, color='red')
axes[0].set_xlabel('C')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Validation Curve - C Parameter')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot gamma parameter validation curve
train_mean_gamma = np.mean(train_scores_gamma, axis=1)
train_std_gamma = np.std(train_scores_gamma, axis=1)
val_mean_gamma = np.mean(val_scores_gamma, axis=1)
val_std_gamma = np.std(val_scores_gamma, axis=1)

axes[1].semilogx(gamma_range, train_mean_gamma, 'o-', color='blue', label='Training accuracy')
axes[1].fill_between(gamma_range, train_mean_gamma - train_std_gamma, 
                    train_mean_gamma + train_std_gamma, alpha=0.2, color='blue')
axes[1].semilogx(gamma_range, val_mean_gamma, 'o-', color='red', label='Validation accuracy')
axes[1].fill_between(gamma_range, val_mean_gamma - val_std_gamma, 
                    val_mean_gamma + val_std_gamma, alpha=0.2, color='red')
axes[1].set_xlabel('γ')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Curve - γ Parameter')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Grid search for best combination
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1]
}

grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Test the best model
best_svm = grid_search.best_estimator_
test_accuracy = best_svm.score(X_test, y_test)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}")
print(f"Test accuracy: {test_accuracy:.3f}")

# Visualize the best model
plot_svm_decision_boundary(X_test, y_test, best_svm, 'Optimized RBF SVM')

## Summary

This notebook demonstrated key concepts in kernel methods:

1. **Linear vs Non-linear Separability**: Understanding when kernel methods are needed
2. **Support Vector Machines**: Maximum margin classification with support vectors
3. **Soft Margins**: Handling non-separable data with regularization parameter C
4. **Kernel Trick**: Implicit mapping to higher dimensions using different kernels
5. **Parameter Tuning**: Effect of C and γ on model complexity and performance
6. **Multi-class Extensions**: One-vs-Rest and One-vs-One strategies
7. **Support Vector Regression**: Extending SVM concepts to regression problems
8. **Hyperparameter Optimization**: Systematic approach to finding optimal parameters

**Key Takeaways:**
- RBF kernel works well for most non-linear problems
- Parameter tuning is crucial for optimal performance
- SVMs are powerful but require careful preprocessing and parameter selection
- Kernel methods provide elegant solutions to non-linear problems through implicit feature mapping